In [43]:
from datasets import list_datasets, load_dataset, load_dataset_builder
from transformers import AutoTokenizer

In [44]:
ds_builder = load_dataset_builder("sst2")

In [45]:
ds_builder.info.description
ds_builder.info.features

{'idx': Value(dtype='int32', id=None),
 'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

In [46]:
train_data = load_dataset("sst2", split="train")
test_data = load_dataset("sst2", split="test")

Found cached dataset sst2 (/Users/vale/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)
Found cached dataset sst2 (/Users/vale/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


In [76]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased",)

In [90]:
tokenizer(train_data[0]["sentence"], padding=True, max_length=10, truncation=True)

{'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [91]:
def tokenization(example):
    return tokenizer(example["sentence"],  padding=True, max_length=128, truncation=True)

dataset = train_data.map(tokenization, batched=True)

  0%|          | 0/68 [00:00<?, ?ba/s]

In [92]:
dataset.format

{'type': None,
 'format_kwargs': {},
 'columns': ['idx',
  'sentence',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask'],
 'output_all_columns': False}

In [93]:
dataset.set_format(type="torch", columns=["idx", "input_ids", "label"])
dataset.format['type']

'torch'

In [94]:
dataset[122]

{'idx': tensor(122),
 'label': tensor(0),
 'input_ids': tensor([  101, 22902,  7982,  1010,  3082,  1011,  4375,  3042,  2100,  1011,
          3110, 15792,  1010,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0])}

In [65]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=128)

In [98]:
from src.model import make_model

ModuleNotFoundError: No module named 'utilities'

In [ ]:
test_model = make_mo